In [5]:
## import 部分  
##%overwritefile
##%file:src/head.py
##%noruncode

from math import exp
from queue import Queue
from threading import Thread

from ipykernel.kernelbase import Kernel
from pexpect import replwrap, EOF
from jinja2 import Environment, PackageLoader, select_autoescape,Template
from abc import ABCMeta, abstractmethod
from typing import List, Dict, Tuple, Sequence
from shutil import copyfile
from plugins.ISpecialID import IStag,IDtag,IBtag,ITag
import pexpect
import signal
import typing 
import typing as t
import re
import signal
import subprocess
import tempfile
import os
import sys
import traceback
import os.path as path
import codecs
import time
import importlib
import importlib.util
import inspect
from plugins._filter2_magics import Magics

[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/head.py created successfully


In [5]:
##全局函数和变量
##%overwritefile
##%file:src/common.py
##%noruncode

##%test_begin

##%test_end

[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/common.py created successfully


In [13]:
##核心内变量与属性  
##%overwritefile
##%file:src/comm_attribute.py
##%noruncode

    ISplugins={"0":[],
         "1":[],
         "2":[],
         "3":[],
         "4":[],
         "5":[],
         "6":[],
         "7":[],
         "8":[],
         "9":[]}
    IDplugins={"0":[],
         "1":[],
         "2":[],
         "3":[],
         "4":[],
         "5":[],
         "6":[],
         "7":[],
         "8":[],
         "9":[]}
    IBplugins={"0":[],
         "1":[],
         "2":[],
         "3":[],
         "4":[],
         "5":[],
         "6":[],
         "7":[],
         "8":[],
         "9":[]}
    silent=None
    jinja2_env = Environment()
    g_rtsps={}
    g_chkreplexit=True



[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/comm_attribute.py created successfully


In [5]:
##解析环境变量参数字符串函数
##%overwritefile
##%file:src/_filter_env.py
##%noruncode
    def _filter_env(self, envstr):
        if envstr is None or len(envstr.strip())<1:
            return os.environ
        # env_dict={}
        argsstr=self.replacemany(self.replacemany(self.replacemany(envstr.strip(),('  '),' '),('= '),'='),' =','=')
        pattern = re.compile(r'([^\s*]*)="(.*?)"|([^\s*]*)=(\'.*?\')|([^\s*]*)=(.[^\s]*)')
        for argument in pattern.findall(argsstr):
            li=list(argument)
            li= [i for i in li if i != '']
            # env_dict[str(li[0])]=li[1]
            os.environ.setdefault(str(li[0]),li[1])
        # envstr=str(str(envstr.split("|")).split("=")).replace(" ","").replace("\'","").replace("\"","").replace("[","").replace("]","").replace("\\","")
        # env_list=envstr.split(",")
        # for i in range(0,len(env_list),2):
        #     os.environ.setdefault(env_list[i],env_list[i+1])
        return os.environ


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/_filter_env.py created successfully


In [7]:
##解析参数字符串函数
##%overwritefile
##%file:src/_filter_dict.py
##%noruncode
    def _filter_dict(self,argsstr):
        if not argsstr or len(argsstr.strip())<1:
            return None
        env_dict={}
        argsstr=replacemany(replacemany(replacemany(argsstr.strip(),('  '),' '),('= '),'='),' =','=')
        pattern = re.compile(r'([^\s*]*)="(.*?)"|([^\s*]*)=(\'.*?\')|([^\s*]*)=(.[^\s]*)')
        for argument in pattern.findall(argsstr):
            li=list(argument)
            li= [i for i in li if i != '']
            env_dict[str(li[0])]=li[1]
        return env_dict

[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/_filter_dict.py created successfully


In [7]:
##清除注释函数
##%overwritefile
##%file:src/cleannotes.py
##%noruncode
    def _is_test_begin(self,line):
        if line==None or line=='':return ''
        return line.strip().startswith('##test_begin') or line.strip().startswith('//test_begin')
    def _is_test_end(self,line):
        if line==None or line=='':return ''
        return line.strip().startswith('##test_end') or line.strip().startswith('//test_end')
    def _is_dqm_begin(self,line):
        if line==None or line=='':return ''
        return line.lstrip().startswith('\"\"\"')
    def _is_dqm_end(self,line):
        if line==None or line=='':return ''
        return line.rstrip().endswith('\"\"\"')
    def _is_sqm_begin(self,line):
        if line==None or line=='':return ''
        return line.lstrip().startswith('\'\'\'')
    def _is_sqm_end(self,line):
        if line==None or line=='':return ''
        return line.rstrip().endswith('\'\'\'')

    def cleannotes(self,line):
        return '' if (not self._is_specialID(line)) and (line.lstrip().startswith('##') or line.lstrip().startswith('//')) else line
    isdqm=False
    def cleandqm(self,line):
        if not self.isdqm:
            istb=self._is_dqm_begin(line)
            if istb: 
                self.isdqm=True
                if len(line.strip())>5:
                    iste=self._is_dqm_end(line)
                    if iste:self.isdqm=False
                return ''
        iste=self._is_dqm_end(line)
        if iste:
            self.isdqm=False
            return ''
        line= "" if self.isdqm else line
        return line
    issqm=False
    def cleansqm(self,line):
        if not self.issqm:
            istb=self._is_sqm_begin(line)
            if istb: 
                self.issqm=True
                if len(line.strip())>5:
                    iste=self._is_sqm_end(line)
                    if iste:self.issqm=False
                return ''
        iste=self._is_sqm_end(line)
        if iste:
            self.issqm=False
            return ''
        line= "" if self.issqm else line
        return line
    istestcode=False
    def cleantestcode(self,line):
        if not self.istestcode:
            istb=self._is_test_begin(line)
            if istb: 
                self.istestcode=True
                if len(line.strip())>5:
                    iste=self._is_test_end(line)
                    if iste:self.istestcode=False
                return ''
        iste=self._is_test_end(line)
        if iste:
            self.istestcode=False
            return ''
        line= "" if self.istestcode else line
        return line
    

rename to:/root/Jupyter/Myjupyter-kernel/src/cleannotes.py
[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/cleannotes.py created successfully


In [4]:
## _is_specialID
##%overwritefile
##%file:src/_is_specialID.py
##%noruncode
def _is_specialID(self,line):
    if line.strip().startswith('##%') or line.strip().startswith('//%'):
        return True
    return False


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/_is_specialID.py created successfully


In [7]:
##kernel_common
##%overwritefile
##%file:src/kernel_common.py
##%noruncode
    usleep = lambda x: time.sleep(x/1000000.0)
    def addkey2dict(self,magics:Dict,key:str):
        if not magics.__contains__(key):
            d={key:[]}
            magics.update(d)
        return magics[key]
    def replacemany(self,our_str, to_be_replaced:str, replace_with:str):
        while (to_be_replaced in our_str):
            our_str = our_str.replace(to_be_replaced, replace_with)
        return our_str

    def _filter_dict(self,argsstr):
        if not argsstr or len(argsstr.strip())<1:
            return None
        env_dict={}
        argsstr=self.replacemany(self.replacemany(self.replacemany(argsstr.strip(),('  '),' '),('= '),'='),' =','=')
        pattern = re.compile(r'([^\s*]*)="(.*?)"|([^\s*]*)=(\'.*?\')|([^\s*]*)=(.[^\s]*)')
        for argument in pattern.findall(argsstr):
            li=list(argument)
            li= [i for i in li if i != '']
            env_dict[str(li[0])]=li[1]
        return env_dict
    def _fileshander(self,files:List,srcfilename,magics)->str:
        index=-1
        fristfile=srcfilename
        try:
            for newsrcfilename in files:
                index=index+1
                newsrcfilename = os.path.join(os.path.abspath(''),newsrcfilename)
                if os.path.exists(newsrcfilename):
                    if magics!=None and len(self.addkey2dict(magics,'overwritefile'))<1:
                        newsrcfilename +=".new.py"
                if not os.path.exists(os.path.dirname(newsrcfilename)) :
                    os.makedirs(os.path.dirname(newsrcfilename))
                if index==0:
                    os.rename(srcfilename,newsrcfilename)
                    fristfile=newsrcfilename
                    files[0]=newsrcfilename
                else:
                    self._write_to_stdout("copy to :"+newsrcfilename+"\n")
                    copyfile(fristfile,newsrcfilename)
        except Exception as e:
                self._log(str(e),2)
        return files[0]

[MyPython] Info:srcfile:/root/Jupyter/Myjupyter-kernel/tmpzlami3qh.py


[MyPython] Info:file /root/Jupyter/Myjupyter-kernel/src/kernel_common.py created successfully


In [14]:
## cleanup_files
##%overwritefile
##%file:src/cleanup_files.py
##%noruncode
    def cleanup_files(self):
        """Remove all the temporary files created by the kernel"""
        # keep the list of files create in case there is an exception
        # before they can be deleted as usual
        for file in self.files:
            if(os.path.exists(file)):
                os.remove(file)



[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/cleanup_files.py created successfully


In [15]:
## new_temp_file
##%overwritefile
##%file:src/new_temp_file.py
##%noruncode
    def new_temp_file(self, **kwargs):
        """Create a new temp file to be deleted when the kernel shuts down"""
        # We don't want the file to be deleted when closed, but only when the kernel stops
        kwargs['delete'] = False
        kwargs['mode'] = 'w'
        file = tempfile.NamedTemporaryFile(**kwargs)
        self.files.append(file.name)
        return file


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/new_temp_file.py created successfully


In [16]:
## _write_display_data
##%overwritefile
##%file:src/_write_display_data.py
##%noruncode
    def _write_display_data(self,mimetype='text/html',contents=""):

        self.send_response(self.iopub_socket, 'display_data', {'data': {mimetype:contents}, 'metadata': {mimetype:{}}})


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/_write_display_data.py created successfully


In [17]:
## _write_to_stdout
##%overwritefile
##%file:src/_write_to_stdout.py
##%noruncode
    def _write_to_stdout(self,contents,magics=None):
        if magics !=None and len(magics['outputtype'])>0:
            self._write_display_data(mimetype=magics['outputtype'],contents=contents)
        else:
            self.send_response(self.iopub_socket, 'stream', {'name': 'stdout', 'text': contents})


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/_write_to_stdout.py created successfully


In [18]:
## _write_to_stderr
##%overwritefile
##%file:src/_write_to_stderr.py
##%noruncode
    def _write_to_stderr(self, contents):
        self.send_response(self.iopub_socket, 'stream', {'name': 'stderr', 'text': contents})


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/_write_to_stderr.py created successfully


In [19]:
## _read_from_stdin
##%overwritefile
##%file:src/_read_from_stdin.py
##%noruncode
    def _read_from_stdin(self):
        return self.raw_input()


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/_read_from_stdin.py created successfully


In [20]:
## readcodefile
##%overwritefile
##%file:src/readcodefile.py
##%noruncode
    def readcodefile(self,filename,spacecount=0):
        filecode=''
        codelist1=None
        if not os.path.exists(filename):
            return ''
        with open(os.path.join(os.path.abspath(''),filename), 'r') as codef1:
            codelist1 = codef1.readlines()
        if len(codelist1)>0:
            for t in codelist1:
                filecode+=' '*spacecount + t
        return filecode


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/readcodefile.py created successfully


In [22]:
## process_output
##%overwritefile
##%file:src/process_output.py
##%noruncode
    def process_output(self, output,magics=None):
        if not self.silent:
            if magics !=None and len(magics['outputtype'])>0:
                self._write_display_data(mimetype=magics['outputtype'],contents=output)
                return
            # Send standard output
            stream_content = {'name': 'stdout', 'text': output}
            self.send_response(self.iopub_socket, 'stream', stream_content)


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/process_output.py created successfully


In [23]:
## _log
##%overwritefile
##%file:src/log_output.py
##%noruncode
    def _log(self, output,level=1,outputtype='text/plain'):
        streamname='stdout'
        if not self.silent:
            prestr=self.kernelinfo+' Info:'
            if level==2:
                prestr=self.kernelinfo+' Warning:'
                streamname='stderr'
            elif level==3:
                prestr=self.kernelinfo+' Error:'
                streamname='stderr'
            else:
                prestr=self.kernelinfo+' Info:'
                streamname='stdout'
            if len(outputtype)>0 and (level!=2 or level!=3):
                self._write_display_data(mimetype=outputtype,contents=prestr+output)
                return
            # Send standard output
            stream_content = {'name': streamname, 'text': prestr+output}
            self.send_response(self.iopub_socket, 'stream', stream_content)

[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/log_output.py created successfully


In [28]:
## send_cmd
##%overwritefile
##%file:src/send_cmd.py
##%noruncode
    def send_cmd(self,pid,cmd):
        try:
            # self._write_to_stdout("send cmd PID:"+pid+"\n cmd:"+cmd)
            # if self.g_rtsps.has_key(pid):
                # self._write_to_stderr("[MyPythonkernel] Info:exist! "+pid+"\n")
            # self.g_rtsps[pid].stdin.write(cmd.encode())
            self.g_rtsps[pid]._write_to_stdout(cmd)
        except Exception as e:
            self._write_to_stderr("[MyPythonkernel] Error:Executable send_cmd error! "+str(e)+"\n")
    
        return


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/send_cmd.py created successfully


In [29]:
## create_jupyter_subprocess
##%overwritefile
##%file:src/create_jupyter_subprocess.py
##%noruncode
    def create_jupyter_subprocess(self, cmd,cwd=None,shell=False,env=None):
        try:
            return RealTimeSubprocess(cmd,
                                  self._write_to_stdout,
                                  self._write_to_stderr,
                                  self._read_from_stdin,cwd,shell,env)
        except Exception as e:
            self._write_to_stdout("RealTimeSubprocess err:"+str(e))
            raise


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/create_jupyter_subprocess.py created successfully


In [31]:
## compile_with_Python2native
##%overwritefile
##%file:src/compile_with_Python2native.py
##%noruncode
    def compile_with_Python2native(self, source_filename, binary_filename, cflags=None, ldflags=None):
        # cflags = ['-std=c89', '-pedantic', '-fPIC', '-shared', '-rdynamic'] + cflags
        # cflags = ['-std=c99', '-Wdeclaration-after-statement', '-Wvla', '-fPIC', '-shared', '-rdynamic'] + cflags
        # cflags = ['-std=iso9899:199409', '-pedantic', '-fPIC', '-shared', '-rdynamic'] + cflags
        # cflags = ['-std=c99', '-pedantic', '-fPIC', '-shared', '-rdynamic'] + cflags
        # cflags = ['-std=c11', '-pedantic', '-fPIC','-pie', '-rdynamic'] + cflags
        outfile=None
        #binary_filename='/root/mytestc.out'
        # if self.linkMaths:
        #     cflags = cflags + ['-lm']
        # if self.wError:
        #     cflags = cflags + ['-Werror']
        # if self.wAll:
        #     cflags = cflags + ['-Wall']
        #if ('-o' not in cflags):
            #binary_filename=cflags[cflags.index('-o')+1]
        #else:
        outfile= ['-o', binary_filename]
        args = ['MyPythonKernel','compile', 'exe', source_filename] + cflags + outfile + ldflags
        # for x in args: self._write_to_stderr(" " + x + " ")
        return self.create_jupyter_subprocess(args)


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/compile_with_Python2native.py created successfully


In [34]:
## _add_main
##%overwritefile
##%file:src/_add_main.py
##%noruncode
    def _add_main(self, magics, code):
        # remove comments
        tmpCode = re.sub(r"//.*", "", code)
        tmpCode = re.sub(r"/\*.*?\*/", "", tmpCode, flags=re.M|re.S)

        x = re.search(r".*\s+main\s*\(", tmpCode)

        if not x:
            code = self.main_head + code + self.main_foot
            magics['cflags'] += ['-lm']

        return magics, code


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/_add_main.py created successfully


In [4]:
## do_shutdown
##%overwritefile
##%file:src/do_shutdown.py
##%noruncode
    def do_shutdown(self, restart):
        self.g_chkreplexit=False
        self.chk_replexit_thread.join()
        # self.onkernelshutdown()
        """Cleanup the created source code files and executables when shutting down the kernel"""
        self.cleanup_files()


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/do_shutdown.py created successfully
